In [333]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import warnings
warnings.filterwarnings('ignore')

In [382]:
tags = ["bbfm", "blu", "dbmc", "dbu"]
tag = tags[3]

In [383]:
features = pd.read_csv('features_'+ tag +'.csv')


In [384]:
df = pd.read_csv('data_proc2.csv')


In [385]:
df[df["target"].isna()].shape


(3263, 8)

In [386]:
features.shape

(10840, 769)

The labels indicating which sentence is positive and negative now go into the `labels` variable

In [387]:
labels = df["target"]

In [388]:
#7613 - 36 удаленных
l = 7577
train_f = features[:l]
test_f = features[l:]

train_l = labels[:l]
test_l = labels[l:]

print(len(train_l))

7577


## Model #2: Train/Test Split
Let's now split our datset into a training set and testing set (even though we're using 2,000 sentences from the SST2 training set).

In [389]:
train_features, test_features, train_labels, test_labels = train_test_split(train_f, train_l, shuffle="False")

In [390]:
parameters = {'C': np.linspace(0.0001, 100, 20)}
grid_search = GridSearchCV(LogisticRegression(), parameters)
grid_search.fit(train_features, train_labels)

print('best parameters: ', grid_search.best_params_)
print('best scrores: ', grid_search.best_score_)

best parameters:  {'C': 73.68423684210526}
best scrores:  0.8141484571456885


We now train the LogisticRegression model. If you've chosen to do the gridsearch, you can plug the value of C into the model declaration (e.g. `LogisticRegression(C=5.2)`).

In [391]:
lr_clf = LogisticRegression( C = grid_search.best_params_['C'])
lr_clf.fit(train_features, train_labels)

LogisticRegression(C=73.68423684210526, class_weight=None, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [392]:
lr_clf.score(test_features, test_labels)

0.8089709762532982

In [393]:
import time
predict = pd.DataFrame(lr_clf.predict_proba(train_f), dtype="float")
submission = pd.concat([pd.DataFrame({"id": df["id"][:l]}), predict], axis=1)
fname = 'LR_train_' + tag + str(time.time()) + 'submission.csv'
submission.to_csv(fname, index=False, header="id,target")

In [394]:
submission.shape

(7577, 3)

In [395]:
predict = pd.DataFrame(lr_clf.predict_proba(test_f), dtype="float")
predict["id"] = np.array(df["id"][l:])
submission = predict
fname = 'LR_test_' + tag + str(time.time()) + 'submission.csv'
submission.to_csv(fname, index=False, header="id,target")
submission.shape

(3263, 3)

In [396]:
submission.info


<bound method DataFrame.info of              0         1     id
0     0.657848  0.342152      0
1     0.479307  0.520693      2
2     0.327140  0.672860      3
3     0.059447  0.940553      9
4     0.003737  0.996263     11
...        ...       ...    ...
3258  0.841127  0.158873  10861
3259  0.068054  0.931946  10865
3260  0.119078  0.880922  10868
3261  0.165236  0.834764  10874
3262  0.325884  0.674116  10875

[3263 rows x 3 columns]>

So our model clearly does better than a dummy classifier. But how does it compare against the best models?

## Proper SST2 scores
For reference, the [highest accuracy score](http://nlpprogress.com/english/sentiment_analysis.html) for this dataset is currently **96.8**. DistilBERT can be trained to improve its score on this task – a process called **fine-tuning** which updates BERT’s weights to make it achieve a better performance in this sentence classification task (which we can call the downstream task). The fine-tuned DistilBERT turns out to achieve an accuracy score of **90.7**. The full size BERT model achieves **94.9**.



And that’s it! That’s a good first contact with BERT. The next step would be to head over to the documentation and try your hand at [fine-tuning](https://huggingface.co/transformers/examples.html#glue). You can also go back and switch from distilBERT to BERT and see how that works.